# Analyzing CIA Factbook Data

In this project, we'll work with data from the [CIA Worl Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics of all the countries on Earth. The factbook contains demographic information like:
- <code style="color:red">population</code> - The population as of <code style="color:red">2015</code>
- <code style="color:red">population_growth</code> - The annual population growth rate, as a percentage.
- <code style="color:red">area</code> - The total land and water area.

You can download the SQLite database, <code style="color:red">factbook.db</code>, from [this GitHub repo](https://github.com/factbook/factbook.sql/releases).

First let's get familiar with the database. Let's check the tables in our database and see the first 5 rows of data of the <code style="color:red">facts</code> table.

In [8]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('factbook.db')
query = "SELECT * FROM sqlite_master WHERE type='table';"
pd.read_sql_query(query, conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


In [11]:
query = "select * from facts limit 5"
pd.read_sql_query(query, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:
- <code style="color:red">name</code> - The name of the country.
- <code style="color:red">population</code> - The country's population.
- <code style="color:red">population_growth</code>- The country's population growth as a percentage.
- <code style="color:red">birth_rate</code> - The country's birth rate, or the number of births a year per 1,000 people.
- <code style="color:red">death_rate</code> - The country's death rate, or the number of death a year per 1,000 people.
- <code style="color:red">area</code> - The country's total area (both land and water).
- <code style="color:red">area_land</code> - The country's land area in square kilometers.
- <code style="color:red">area_water</code> - The country's waterarea in square kilometers.


# Summary Statistics

Está completo o passo 1. Falta completar o passo 2